# PEC2

![lego](https://kaggle2.blob.core.windows.net/datasets-images/1599/2846/759e9518355eabd33bd70f907e4f7992/dataset-cover.jpg)


Vamos a trabajar sobre un dataset de LEGO, en el que tenemos diferentes ficheros/tablas.
Éste es un esquema de las tablas que representan los CSV de datos que tenemos:

![diagram](downloads_schema.png)

## 1 Carga de Datos
En la tabla `sets` tenemos los diferentes sets de lego, cada uno con su identificador, año y su "temática".
En la tabla `themes` tenemos las diferentes temáticas de todos los sets de lego.
Carga ambos DataFrames, y modifica el esquema si lo crees oportuno.

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='app15')
sqlContext = pyspark.SQLContext(sc)

In [8]:
colors =  sqlContext.read.option("header", "true").format("csv").load("data/colors.csv")
inventories = sqlContext.read.option("header", "true").format("csv").load("data/inventories.csv")
inventory_sets = sqlContext.read.option("header", "true").format("csv").load("data/inventory_sets.csv")
part_categories = sqlContext.read.option("header", "true").format("csv").load("data/part_categories.csv")
parts = sqlContext.read.option("header", "true").format("csv").load("data/parts.csv")
sets = sqlContext.read.option("header", "true").format("csv").load("data/sets.csv")
themes = sqlContext.read.option("header", "true").format("csv").load("data/themes.csv")

## 2 Joins
Ahora queremos extraer **sólo** las siguientes columnas:
 - el identificador del set (`set_num`)
 - la temática del set (columna `name` de `themes`)
 - el nombre del set (columna `name` de `sets`)
 
De todos los sets que contengan "x-wing" en su nombre, y sean posteriores a 2011.
Ten en cuenta que "x-wing" puede aparecer en cualquier posición del nombre del set, y con cualquier composición de mayúsculas o minúsculas. (es decir, tienes que detectar apariciones de `x-wing`, `X-Wing`, `X-wing`...etc).

Ten en cuenta que sólo queremos esas 3 columnas en el resultado.

In [41]:
sets.createOrReplaceTempView("sets")
themes.createOrReplaceTempView("themes")
inventory_sets.createOrReplaceTempView("inventory_sets")

In [40]:
sqlContext.sql("select a.set_num,a.name as name_sets,b.name as name_theme from sets a left join themes b on a.theme_id=b.id where lower(a.name) like ('%x-wing%') and a.year<2011").show(1000)

+-------+--------------------+--------------------+
|set_num|           name_sets|          name_theme|
+-------+--------------------+--------------------+
|30051-1|X-wing Fighter - ...|Star Wars Episode...|
| 4484-1|X-wing Fighter & ...|                Mini|
| 4502-1|X-wing Fighter (D...|Star Wars Episode...|
| 4502-2|X-wing Fighter (D...|Star Wars Episode...|
| 6212-1|      X-wing Fighter|Star Wars Episode...|
|65145-1|X-wing Fighter / ...|Star Wars Episode...|
|66221-1|X-wing Fighter an...|Star Wars Episode...|
| 6963-1|X-wing Fighter - ...|                Mini|
| 6963-2|X-wing Fighter - ...|                Mini|
| 7140-1|      X-wing Fighter|Star Wars Episode...|
| 7142-1|X-wing Fighter (r...|Star Wars Episode...|
| 7191-1|X-wing Fighter - UCS|Star Wars Episode...|
+-------+--------------------+--------------------+



## 3 Más joins
Ahora queremos que nuestro resultado incluya todos los sets de la temática Star wars (id = `158`), y además, para aquellos sets para los que exista información de "inventario" (tabla `inventory_sets`) queremos que aparezca también en el resultado.

Es decir, el resultado debe incluír **todos los sets cuyo theme_id sea 158** y sólo las siguientes columnas:
 - `set_num`
 - `name` (nombre del set)
 - `inventory_id`
 - `quantity`
 
Cuando no exista información de `inventory_id` ó `quantity` para un set determinado, queremos que los valores sean `null`.

In [48]:
sqlContext.sql("select a.set_num,a.name as name_sets,c.inventory_id,c.quantity \
               from sets a \
               left join themes b on a.theme_id=b.id \
               left join inventory_sets c on a.set_num = c.set_num \
               where b.id=158").show(10000)

+-----------+--------------------+------------+--------+
|    set_num|           name_sets|inventory_id|quantity|
+-----------+--------------------+------------+--------+
|    10018-1|          Darth Maul|        null|    null|
|    11912-1|LEGO Star Wars: B...|        null|    null|
|  2853835-1|White Boba Fett F...|        null|    null|
|    30058-1|                STAP|        null|    null|
|    30059-1|                 MTT|        null|    null|
|    30246-1|    Imperial Shuttle|        null|    null|
|    30247-1| ARC-170 Starfighter|        null|    null|
|    30276-1|First Order Speci...|        null|    null|
|    30602-1|First Order Storm...|        null|    null|
|    30611-1|               R2-D2|        null|    null|
|     4481-1|      Hailfire Droid|       11344|       1|
|  4547551-1|Darth Vader 10 Ye...|        null|    null|
|  5000067-1|  Star Wars Sith Kit|        null|    null|
|  5001137-1|Battle Pack Colle...|        null|    null|
|  5001308-1|The Old Republic .